<a href="https://colab.research.google.com/github/PatoJosefo/API-NEXUS/blob/main/Colab_Unificado_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================
# CELL 1: Imports
# ===============================
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output,Javascript
from google.colab import drive
import seaborn as sns # Import seaborn for easier grouped plots
import warnings

# Optional: Ignore specific warnings if they are noisy, e.g., FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print("Cell 1/11: Libraries imported.")

In [ ]:
# ===============================
# CELL 2: Mount Google Drive & Define Path
# ===============================
# Mount Google Drive to access your files
# You might need to authorize Colab access the first time
try:
    drive.mount('/content/drive', force_remount=True) # force_remount can be useful
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    print("Please ensure you have authorized access and the Drive path is correct.")

# !!! IMPORTANT !!!
# UPDATE THIS PATH if your 'API1S' folder is located elsewhere in your Google Drive
DRIVE_BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/API1S/"
print(f"Using base path for data: {DRIVE_BASE_PATH}")
print("--> Please VERIFY this path points to your folder containing the CSV files <--")

print("\nCell 2/11: Drive mount attempted and base path set.")

In [ ]:
# ===============================
# CELL 3: Load Auxiliary Data
# ===============================
# Load helper CSV files (Municipalities and NCM/SH codes)

print("Loading auxiliary data...")
# Define file paths using the base path
municipios_file = f"{DRIVE_BASE_PATH}UF_MUN.csv"
ncm_sh_file = f"{DRIVE_BASE_PATH}NCM_SH.csv"

try:
    print(f"Attempting to load: {municipios_file}")
    municipios_df = pd.read_csv(
        municipios_file,
        delimiter=';',
        encoding='latin-1'
    )
    # Clean column names immediately after loading
    municipios_df.columns = municipios_df.columns.str.strip()
    print(f"  Loaded UF_MUN.csv ({len(municipios_df)} rows)")

    print(f"Attempting to load: {ncm_sh_file}")
    ncm_sh_df = pd.read_csv(
        ncm_sh_file,
        delimiter=';',
        encoding='latin-1'
    )
    # Clean column names immediately after loading
    ncm_sh_df.columns = ncm_sh_df.columns.str.strip()
    print(f"  Loaded NCM_SH.csv ({len(ncm_sh_df)} rows)")

    # --- Data Cleaning for Auxiliary DataFrames ---
    print("Cleaning auxiliary data...")
    # Ensure merge keys are of compatible types and handle errors
    municipios_df['CO_MUN_GEO'] = pd.to_numeric(municipios_df['CO_MUN_GEO'], errors='coerce')
    rows_before = len(municipios_df)
    municipios_df.dropna(subset=['CO_MUN_GEO'], inplace=True) # Remove rows where CO_MUN_GEO is not numeric
    print(f"  Municipios: Dropped {rows_before - len(municipios_df)} rows with invalid CO_MUN_GEO.")
    municipios_df['CO_MUN_GEO'] = municipios_df['CO_MUN_GEO'].astype(int)

    ncm_sh_df['CO_SH4'] = pd.to_numeric(ncm_sh_df['CO_SH4'], errors='coerce')
    rows_before = len(ncm_sh_df)
    ncm_sh_df.dropna(subset=['CO_SH4'], inplace=True) # Remove rows where CO_SH4 is not numeric
    print(f"  NCM_SH: Dropped {rows_before - len(ncm_sh_df)} rows with invalid CO_SH4.")
    ncm_sh_df['CO_SH4'] = ncm_sh_df['CO_SH4'].astype(int)

    print("Auxiliary data loaded and cleaned successfully.")
    # Display info to verify (optional but recommended)
    # print("\n_Municipios DataFrame Info:_")
    # municipios_df.info()
    # print("\n_NCM SH DataFrame Info:_")
    # ncm_sh_df.info()

except FileNotFoundError as e:
    print(f"\n--- ERROR ---")
    print(f"Error loading auxiliary file: {e}")
    print(f"Please ensure '{e.filename}' exists at the specified path: {DRIVE_BASE_PATH}")
    print(f"Check the DRIVE_BASE_PATH in Cell 2 and the file names.")
    print(f"-------------\n")
    # Raise an error or define empty dataframes to prevent later failures
    municipios_df = pd.DataFrame()
    ncm_sh_df = pd.DataFrame()
except Exception as e:
    print(f"\n--- ERROR ---")
    print(f"An unexpected error occurred loading auxiliary data: {e}")
    print(f"-------------\n")
    municipios_df = pd.DataFrame()
    ncm_sh_df = pd.DataFrame()


print("\nCell 3/11: Auxiliary data loading attempted.")

In [ ]:
# ===============================
# CELL 4: Data Loading Function Definition
# ===============================
# Function to load and preprocess yearly export data

def carregar_dados_anuais(anos_lista):
    """
    Loads export data for specified years, filters for SP, cleans, and preprocesses.

    Args:
        anos_lista (list): A list of strings representing the years to load (e.g., ['2019', '2020']).

    Returns:
        dict: A dictionary where keys are years (str) and values are preprocessed pandas DataFrames for SP.
              Returns None if auxiliary data wasn't loaded properly first.
    """
    dados = {}
    print(f"Starting data loading for years: {anos_lista}...")

    # --- Pre-check: Ensure auxiliary dataframes exist ---
    # Use 'globals()' to check if the variables exist in the global scope
    if 'municipios_df' not in globals() or municipios_df.empty or \
       'ncm_sh_df' not in globals() or ncm_sh_df.empty:
         print("\n--- ERROR ---")
         print("Auxiliary data (municipios_df or ncm_sh_df) not loaded successfully or is empty.")
         print("Please ensure Cell 3 ran without errors before running this cell.")
         print("-------------\n")
         return None # Indicate critical failure


    for ano in anos_lista:
        file_path = f"{DRIVE_BASE_PATH}EXP_{ano}_MUN.csv" # Use the defined base path from Cell 2
        print(f"Attempting to load: {file_path}")
        try:
            # Load with specified dtypes, then clean columns
            df = pd.read_csv(
                file_path,
                delimiter=';',
                dtype={'KG_LIQUIDO': str, 'VL_FOB': str, 'CO_MUN': str, 'SH4': str}, # Read key columns as string initially
                encoding='latin-1' # Keep encoding consistent
            )
            df.columns = df.columns.str.strip() # Clean column names
            print(f"  Loaded {ano}, processing...")

            # --- Filter for SP Municipalities ---
            df_sp = pd.DataFrame() # Initialize empty DataFrame
            if 'SG_UF_MUN' in df.columns:
                 # Primary method: Use SG_UF_MUN if available
                 df_sp = df[df['SG_UF_MUN'].str.contains('SP', case=False, na=False)].copy() # case=False for robustness
                 print(f"  Filtered for SP using 'SG_UF_MUN': {len(df_sp)} rows found.")
            elif 'CO_MUN' in df.columns and 'municipios_df' in globals() and not municipios_df.empty:
                 # Fallback method: Use CO_MUN and the auxiliary municipios_df
                 print("  'SG_UF_MUN' not found or unusable. Filtering SP based on 'CO_MUN' using auxiliary data.")
                 sp_codes = municipios_df[municipios_df['SG_UF'] == 'SP']['CO_MUN_GEO'].unique() # Get codes for SP from aux data
                 # Ensure CO_MUN in df is numeric before filtering
                 df['CO_MUN_temp'] = pd.to_numeric(df['CO_MUN'], errors='coerce')
                 df_filtered_numeric = df.dropna(subset=['CO_MUN_temp']).copy() # Work on rows with valid numeric CO_MUN
                 df_filtered_numeric['CO_MUN_temp'] = df_filtered_numeric['CO_MUN_temp'].astype(int)
                 df_sp = df_filtered_numeric[df_filtered_numeric['CO_MUN_temp'].isin(sp_codes)].copy()
                 df_sp = df_sp.drop(columns=['CO_MUN_temp']) # Remove temporary column
                 print(f"  Filtered for SP using 'CO_MUN': {len(df_sp)} rows found.")
            else:
                 # If neither method works
                 print("  --- WARNING ---: Could not filter for SP municipalities.")
                 print("    Reason: 'SG_UF_MUN' column missing or unusable, AND ('CO_MUN' column missing or auxiliary 'municipios_df' unavailable/empty).")
                 print("    Proceeding with unfiltered data for this year, which may include non-SP locations.")
                 df_sp = df.copy() # Use the full dataframe as a last resort

            if df_sp.empty:
                print(f"  No SP data found (or filtered) for {ano}. Skipping further processing for this year.")
                continue # Skip to the next year

            # --- Convert Numeric and Identifier Columns ---
            # Convert with error coercion, allowing us to drop problematic rows later
            df_sp['KG_LIQUIDO'] = pd.to_numeric(
                df_sp['KG_LIQUIDO'].str.replace(',', '.', regex=False), errors='coerce'
            )
            df_sp['VL_FOB'] = pd.to_numeric(
                df_sp['VL_FOB'].str.replace(',', '.', regex=False), errors='coerce'
            )
            # Convert key identifiers, coercing errors
            df_sp['CO_MUN'] = pd.to_numeric(df_sp['CO_MUN'], errors='coerce')
            df_sp['SH4'] = pd.to_numeric(df_sp['SH4'], errors='coerce')


            # --- Drop Rows with Invalid Essential Data ---
            # Essential columns for analysis: CO_MUN, SH4, KG_LIQUIDO, VL_FOB
            essential_cols = ['CO_MUN', 'SH4', 'KG_LIQUIDO', 'VL_FOB']
            initial_rows = len(df_sp)
            df_sp.dropna(subset=essential_cols, inplace=True)
            dropped_rows = initial_rows - len(df_sp)
            if dropped_rows > 0:
                print(f"  Dropped {dropped_rows} rows due to missing/invalid essential values in: {', '.join(essential_cols)}.")


            # --- Convert Identifiers to Integer Type ---
            # Do this *after* dropping NaNs introduced by 'coerce'
            if not df_sp.empty:
                df_sp['CO_MUN'] = df_sp['CO_MUN'].astype(int)
                df_sp['SH4'] = df_sp['SH4'].astype(int)
            else:
                print(f"  No valid data remained for {ano} after cleaning essential columns.")
                continue # Skip to next year if cleaning removed all data


            # --- Store Processed Data ---
            if not df_sp.empty:
                 dados[ano] = df_sp
                 print(f"  Successfully processed and stored data for {ano}. Final shape: {df_sp.shape}")
                 # Optional: Display sample/info for verification
                 # print(f"  {ano} Data Sample (first 5 rows):\n", df_sp.head())
                 # print(f"  {ano} Data Types:\n", df_sp.dtypes)
            # No else needed here, already handled by the 'continue' statements above

        except FileNotFoundError:
            print(f"  --- WARNING ---: File not found for year {ano} at {file_path}. Skipping this year.")
        except Exception as e:
            print(f"  --- ERROR ---: An unexpected error occurred loading or processing data for year {ano}: {e}")
            # Optionally, you might want to store a placeholder or skip depending on severity
            # For now, we just print the error and continue to the next year

    # --- Final Check ---
    if not dados:
        print("\n--- WARNING ---: No yearly data could be loaded successfully.")
        print("  Please check file paths, file existence, and potential errors in the logs above.")
    else:
        print(f"\nFinished loading yearly data. Successfully loaded years: {list(dados.keys())}")

    return dados


print("Cell 4/11: Data loading function 'carregar_dados_anuais' defined.")

In [ ]:
# ===============================
# CELL 5: Execute Data Loading & Prepare Lists
# ===============================
# Define the years you want to load
anos_para_carregar = ['2019', '2020', '2021', '2022', '2023', '2024'] # Adjust as needed
# NOTE: '2024' data might be partial or unavailable, add it if you have the file.

# --- Load the data using the function ---
# This dictionary will hold the DataFrames, keyed by year
dados_ano = carregar_dados_anuais(anos_para_carregar)

# --- Prepare SP Municipality List for Dropdowns ---
municipios_sp_list = ['-- N/A --'] # Default placeholder if loading fails
municipios_sp = pd.DataFrame() # Initialize empty df for SP subset of municipios_df

# Check if data loading was successful and auxiliary data exists
if dados_ano and 'municipios_df' in globals() and not municipios_df.empty:
    print("\nPreparing SP Municipality list for dropdowns...")
    # Get all unique CO_MUN codes present across ALL loaded years FOR SP
    all_sp_mun_codes = set()
    for ano, df_ano in dados_ano.items():
        if df_ano is not None and not df_ano.empty and 'CO_MUN' in df_ano.columns:
             all_sp_mun_codes.update(df_ano['CO_MUN'].unique())
        # else:
        #      print(f" Note: No data or CO_MUN column in dados_ano['{ano}'] to extract codes from.")

    if all_sp_mun_codes:
        print(f" Found {len(all_sp_mun_codes)} unique SP CO_MUN codes in the loaded data.")
        # Filter the main municipios_df based on these codes found in the actual data
        # AND ensure they are marked as SP ('SG_UF') in the auxiliary file for robustness
        try:
            # Ensure CO_MUN_GEO in municipios_df is integer for matching
            municipios_df['CO_MUN_GEO'] = municipios_df['CO_MUN_GEO'].astype(int) # Should be done in cell 3, but safe check
            municipios_sp = municipios_df[
                (municipios_df['CO_MUN_GEO'].isin(all_sp_mun_codes)) &
                (municipios_df['SG_UF'] == 'SP') # Explicitly check for SP state code
            ].copy()

            # Create the list of names from the filtered DataFrame
            if not municipios_sp.empty:
                 municipios_sp_list = sorted(municipios_sp['NO_MUN'].astype(str).unique().tolist())
                 print(f" Successfully created list of {len(municipios_sp_list)} unique SP municipality names found in data.")
                 if not municipios_sp_list: # Should not happen if municipios_sp is not empty, but safety check
                      print(" Warning: Filtering resulted in an empty name list, though SP municipios_df subset was not empty. Check 'NO_MUN' column.")
                      municipios_sp_list = ['-- N/A --']
            else:
                 print(" Warning: No municipalities from the loaded data matched SP entries in the auxiliary file.")
                 municipios_sp_list = ['-- N/A --']

        except Exception as e:
            print(f" Error during filtering/creation of SP municipality list: {e}")
            municipios_sp_list = ['-- N/A --'] # Reset on error
            municipios_sp = pd.DataFrame() # Reset SP dataframe on error

    else:
        print(" Warning: No SP municipality codes were extracted from the loaded yearly data. Cannot create municipality list.")
        municipios_sp_list = ['-- N/A --']
        # Ensure municipios_sp is an empty DataFrame
        municipios_sp = pd.DataFrame(columns=municipios_df.columns if 'municipios_df' in globals() else [])

else:
    print("\n--- WARNING ---")
    if not dados_ano:
        print("Yearly data ('dados_ano') could not be loaded.")
    if 'municipios_df' not in globals() or municipios_df.empty:
        print("Auxiliary data ('municipios_df') is not available or empty.")
    print("Cannot prepare SP municipality list. Dropdowns will be limited.")
    municipios_sp_list = ['-- N/A --']
    # Ensure municipios_sp is defined, even if empty
    municipios_sp = pd.DataFrame(columns=municipios_df.columns if 'municipios_df' in globals() else [])


# Display first few options to verify
print(f"\nFinal SP Municipality List for Dropdowns (first 20): {municipios_sp_list[:20]}")
if municipios_sp_list == ['-- N/A --']:
    print("  -> Dropdowns will show N/A. Please check data loading steps (Cells 2-5).")

print("\nCell 5/11: Yearly data loading executed, SP list prepared.")

In [ ]:
# ===============================
# CELL 6: Widget Definitions
# ===============================
# Define all interactive widgets (dropdowns, buttons, output area)

print("Defining widgets...")

# --- Determine Valid Options ---
# Use keys from the loaded data dictionary for years
valid_anos = list(dados_ano.keys()) if dados_ano else ['Select Year']
# Use the generated list for municipalities
valid_municipios = municipios_sp_list if municipios_sp_list and municipios_sp_list[0] != '-- N/A --' else ['Select Municipality']

# --- Set Sensible Default Values ---
default_ano = valid_anos[0] if valid_anos and valid_anos[0] != 'Select Year' else None
default_municipio1 = valid_municipios[0] if valid_municipios and valid_municipios[0] != 'Select Municipality' else None
# Try to select a *different* second municipality if possible
default_municipio2 = valid_municipios[1] if len(valid_municipios) > 1 and valid_municipios[0] != 'Select Municipality' else default_municipio1

# --- Create Widgets ---

# Year Dropdown
ano_dropdown = widgets.Dropdown(
    options=valid_anos,
    value=default_ano,
    description='Ano:',
    disabled=not default_ano, # Disable if no valid years loaded
    style={'description_width': 'initial'} # Adjust width to fit description
)

# Single Municipality Dropdown (for single view)
municipio_dropdown = widgets.Dropdown(
    options=valid_municipios,
    value=default_municipio1,
    description='Município:',
    disabled=not default_municipio1, # Disable if no valid municipalities
    style={'description_width': 'initial'}
)

# Metric Dropdown
metrica_dropdown = widgets.Dropdown(
    options=['Quilogramas', 'Valor Agregado'], # The two analysis metrics
    value='Quilogramas', # Default metric
    description='Métrica:',
    style={'description_width': 'initial'}
)

# Button for Top 5 Exporting Municipalities (overall SP)
top5_municipios_button = widgets.Button(
    description='Top 5 Municípios Exportadores (SP)',
    button_style='info', # 'info', 'success', 'warning', 'danger' or ''
    tooltip='Mostrar os 5 municípios de SP que mais exportaram no ano selecionado, pela métrica escolhida.',
    icon='list-ol', # FontAwesome icon name
    disabled=not default_ano # Disable if no data loaded (no year selected)
)

# --- Widgets for Comparison View ---

# Municipality 1 Dropdown (for comparison)
municipio1_dropdown = widgets.Dropdown(
    options=valid_municipios,
    value=default_municipio1,
    description='Comparar Município 1:',
    disabled=not default_municipio1,
    style={'description_width': 'initial'}
)

# Municipality 2 Dropdown (for comparison)
municipio2_dropdown = widgets.Dropdown(
    options=valid_municipios,
    value=default_municipio2, # Use potentially different default
    description='Comparar Município 2:',
    disabled=not default_municipio2, # Check second default specifically
    style={'description_width': 'initial'}
)

# Button to trigger comparison plot
compare_button = widgets.Button(
    description='Comparar Top 5 Produtos',
    button_style='success',
    tooltip='Comparar os top 5 produtos (pela métrica) dos dois municípios selecionados.',
    icon='exchange-alt', # FontAwesome icon name
    disabled=not default_municipio1 or not default_municipio2 # Disable if either default is bad/unavailable
)

# --- Output Area Widget ---
# This widget will hold the plots and messages generated by the handlers
output_area = widgets.Output(layout={'border': '1px solid black', 'padding': '10px'}) # Add border for visibility

print("Widgets defined successfully.")
print("--> Widgets will be displayed in Cell 11 after event handlers are set up.")

print("\nCell 6/11: Widgets defined.")

In [ ]:
# ===============================
# CELL 7: Helper & Calculation Functions
# ===============================
# Define functions used for data lookup and metric calculations

# --- Helper function to get CO_MUN_GEO from NO_MUN ---
def get_co_mun(nome_municipio):
    """Looks up the CO_MUN_GEO code for a given municipality name using the 'municipios_sp' DataFrame."""
    # Check if the necessary dataframe is available
    if 'municipios_sp' not in globals() or municipios_sp.empty:
         print(f"Warning: 'municipios_sp' DataFrame is not available or empty. Cannot look up CO_MUN for '{nome_municipio}'.")
         return None

    try:
        # Ensure consistent data types for matching
        municipios_sp['NO_MUN'] = municipios_sp['NO_MUN'].astype(str)
        nome_municipio_str = str(nome_municipio) # Ensure input is string

        # Perform the lookup
        result = municipios_sp[municipios_sp['NO_MUN'] == nome_municipio_str]['CO_MUN_GEO']

        if not result.empty:
            # Return the first match if found (should be unique in municipios_sp)
            return int(result.iloc[0]) # Return as integer
        else:
            # If not found in the filtered SP list, log an error/warning
            print(f"Warning: CO_MUN code not found for municipality '{nome_municipio_str}' in the filtered 'municipios_sp' list.")
            # Optional: You could add a fallback search in the main 'municipios_df' here if needed,
            # but it might indicate an issue with the SP filtering logic in Cell 5.
            return None # Indicate failure clearly
    except Exception as e:
        print(f"Error during CO_MUN lookup for '{nome_municipio}': {e}")
        return None


# --- Calculation function for a single municipality's top 5 products ---
def calcular_top_produtos(df_ano, co_mun, metrica):
    """
    Calculates the top 5 products for a given municipality based on the selected metric.

    Args:
        df_ano (pd.DataFrame): The DataFrame containing export data for a specific year.
        co_mun (int): The CO_MUN_GEO code of the municipality.
        metrica (str): The metric to use ('Quilogramas' or 'Valor Agregado').

    Returns:
        pd.DataFrame: A DataFrame with the top 5 products, their metric values,
                      and descriptions, or an empty DataFrame if data is insufficient or errors occur.
    """
    # --- Input Validation ---
    if df_ano is None or df_ano.empty:
        # print(f"Debug: Input DataFrame for year is empty or None for CO_MUN {co_mun}.") # Debugging info
        return pd.DataFrame()

    if co_mun is None:
        print("Warning: Invalid CO_MUN (None) provided to calcular_top_produtos.")
        return pd.DataFrame()

    # Check if auxiliary NCM/SH data is available
    if 'ncm_sh_df' not in globals() or ncm_sh_df.empty:
        print("Error: ncm_sh_df (product descriptions) not loaded. Cannot calculate top products.")
        return pd.DataFrame()

    # --- Filter data for the specific municipality ---
    df_mun = df_ano[df_ano['CO_MUN'] == co_mun].copy()

    if df_mun.empty:
        # print(f"Debug: No data found in df_ano for CO_MUN {co_mun}.") # More detailed debug if needed
        return pd.DataFrame() # Return empty if no data rows for this municipality in the year's df

    # --- Calculate Metric based on Selection ---
    calculated_metric_col = None # Store the name of the column holding the calculated metric value
    df_metricas = pd.DataFrame() # Initialize result DataFrame

    if metrica == 'Quilogramas':
        coluna_orig = 'KG_LIQUIDO'
        if coluna_orig not in df_mun.columns:
             print(f"Error: Required column '{coluna_orig}' not found in data for CO_MUN {co_mun}.")
             return pd.DataFrame() # Cannot proceed
        # Group by product code (SH4) and sum the kilograms
        df_metricas = df_mun.groupby('SH4')[coluna_orig].sum().reset_index()
        calculated_metric_col = coluna_orig # The column to rank by is KG_LIQUIDO itself

    elif metrica == 'Valor Agregado':
        # Check for required columns
        if not all(col in df_mun.columns for col in ['VL_FOB', 'KG_LIQUIDO']):
             print(f"Error: Required columns 'VL_FOB' or 'KG_LIQUIDO' not found for CO_MUN {co_mun}.")
             return pd.DataFrame() # Cannot proceed

        # Group by product code (SH4) and sum FOB value and kilograms
        df_metricas_agg = df_mun.groupby('SH4').agg(
            Total_FOB=('VL_FOB', 'sum'),
            Total_KG=('KG_LIQUIDO', 'sum')
        ).reset_index()

        # Filter out rows where Total_KG is zero or very close to zero to avoid division errors
        epsilon = 1e-9 # Use a very small number to handle potential floating point issues
        df_metricas = df_metricas_agg[df_metricas_agg['Total_KG'] > epsilon].copy()

        if df_metricas.empty:
            # print(f"Debug: No products with non-zero KG found for CO_MUN {co_mun} to calculate Valor Agregado.") # Debug info
            return pd.DataFrame() # Return empty if no valid data after filtering

        # Calculate the Valor Agregado (FOB / KG)
        df_metricas['Valor_Agregado'] = df_metricas['Total_FOB'] / df_metricas['Total_KG']
        calculated_metric_col = 'Valor_Agregado' # This is the column to rank by
    else:
        # Handle unknown metric input
        print(f"Error: Unknown metric '{metrica}' provided to calcular_top_produtos.")
        return pd.DataFrame()


    # --- Get Top 5 Products ---
    # Check if the calculation was successful and the metric column exists
    if calculated_metric_col is None or calculated_metric_col not in df_metricas.columns:
         print(f"Error: Metric column '{calculated_metric_col}' was not calculated correctly or is missing for CO_MUN {co_mun}.")
         return pd.DataFrame()

    # Ensure there's data to rank
    if df_metricas.empty:
        # This case is likely handled above (e.g., no non-zero KG), but good failsafe
        return pd.DataFrame()

    # Get the top 5 rows based on the calculated metric column
    top5 = df_metricas.nlargest(5, calculated_metric_col)

    # --- Add Product Descriptions ---
    try:
        # Ensure merge keys (SH4 codes) are of compatible integer types
        top5['SH4'] = top5['SH4'].astype(int)
        # ncm_sh_df['CO_SH4'] should already be int from Cell 3 cleaning
        if 'CO_SH4' not in ncm_sh_df.columns:
            print("Error: 'CO_SH4' column missing in ncm_sh_df. Cannot merge product names.")
            top5['NO_SH4_POR'] = 'Merge Error (No CO_SH4)'
        else:
             ncm_sh_df['CO_SH4'] = ncm_sh_df['CO_SH4'].astype(int) # Ensure type just before merge
             top5 = top5.merge(
                 ncm_sh_df[['CO_SH4', 'NO_SH4_POR']], # Select only needed columns from ncm_sh_df
                 left_on='SH4',
                 right_on='CO_SH4',
                 how='left' # Use 'left' merge to keep all top 5 products even if no description is found
             )
    except Exception as e:
        print(f"Error merging top 5 data with NCM SH descriptions for CO_MUN {co_mun}: {e}")
        top5['NO_SH4_POR'] = 'Merge Error' # Add placeholder name if merge fails

    # --- Final Formatting ---
    # Create a descriptive 'Produto' label (Description (SH4 Code))
    # Handle cases where the merge might have failed or description is missing
    top5['NO_SH4_POR'] = top5['NO_SH4_POR'].fillna('Descrição Desconhecida')
    # Ensure SH4 is string type for concatenation, truncate description
    top5['Produto'] = top5['NO_SH4_POR'].astype(str).str[:40] + ' (' + top5['SH4'].astype(str) + ')'
    # Add the municipality code back for reference if needed later
    top5['CO_MUN'] = co_mun

    # Return only the necessary columns for plotting/display
    # Include the specific metric column used for ranking
    columns_to_return = ['CO_MUN', 'SH4', 'Produto', calculated_metric_col]
    # Add other potentially useful columns if they exist from aggregation (like Total_FOB, Total_KG for Valor Agregado)
    if metrica == 'Valor Agregado' and all(c in top5.columns for c in ['Total_FOB', 'Total_KG']):
        columns_to_return.extend(['Total_FOB', 'Total_KG'])

    return top5[columns_to_return]


print("Cell 7/11: Helper and calculation functions defined.")

In [ ]:
# ===============================
# CELL 8: Plotting Functions (Corrected)
# ===============================
# Functions to generate the different types of plots using Matplotlib and Seaborn

# --- Plot Top 5 Products for a SINGLE Municipality ---
def criar_grafico_municipio_unico(dados, municipio_nome, ano, metrica):
    """Generates and displays a horizontal bar chart for the top 5 products of a single municipality."""
    print(f"--- Generating Plot: Single Municipality ---")
    print(f"  Municipality: {municipio_nome}")
    print(f"  Year: {ano}")
    print(f"  Metric: {metrica}")

    # --- Get Municipality Code ---
    co_mun = get_co_mun(municipio_nome)
    if co_mun is None:
        # Error message handled by get_co_mun, just display placeholder plot
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, f"Erro: Código não encontrado para o município '{municipio_nome}'.",
                 ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    # --- Validate Year and Data Availability ---
    if ano not in dados or dados[ano] is None or dados[ano].empty:
        errmsg = f"Erro: Dados não disponíveis para o ano {ano}."
        print(f"  {errmsg}")
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    # --- Calculate Top 5 Products ---
    df_ano = dados[ano]
    top5_data = calcular_top_produtos(df_ano, co_mun, metrica) # Use the calculation function from Cell 7

    # --- Handle No Data Case ---
    if top5_data.empty:
        infomsg = f"Sem dados de exportação Top 5 encontrados para:\n" \
                  f"Município: {municipio_nome} (Código: {co_mun})\n" \
                  f"Ano: {ano}\nMétrica: {metrica}"
        # FIX: Perform replacement before f-string
        log_msg = infomsg.replace('\n', ' ')
        print(f"  Info: {log_msg}") # Log info
        plt.figure(figsize=(10, 4)) # Make figure slightly larger for multi-line text
        plt.text(0.5, 0.5, infomsg, ha='center', va='center', fontsize=12, color='blue')
        plt.axis('off') # Hide axes for message display
        plt.show()
        return

    # --- Prepare for Plotting ---
    # Determine the column name holding the metric value based on the selection
    coluna_plot = 'KG_LIQUIDO' if metrica == 'Quilogramas' else 'Valor_Agregado'
    # Double-check if this column actually exists in the result (should always, given calcular_top_produtos)
    if coluna_plot not in top5_data.columns:
        errmsg = f"Erro Interno: Coluna da métrica '{coluna_plot}' não encontrada nos dados calculados."
        print(f"  {errmsg}")
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    # Set plot labels based on metric
    eixo_x_label = 'Massa Líquida (Kg)' if metrica == 'Quilogramas' else 'Valor Agregado (USD/kg)'
    titulo = f'Top 5 Produtos Exportados - {municipio_nome} ({ano})'

    # --- Create the Plot ---
    plt.figure(figsize=(12, 7)) # Adjust figure size as needed for readability

    # Sort data for consistent plotting order (e.g., highest value bar at the top)
    # For horizontal bar (barh), sorting ascending means highest value is at the top
    plot_data = top5_data.sort_values(coluna_plot, ascending=True)

    # Create the horizontal bar plot using Seaborn
    sns.barplot(data=plot_data, y='Produto', x=coluna_plot, color='cornflowerblue', orient='h') # Changed color

    # Customize plot appearance
    plt.title(titulo, fontsize=16, pad=15) # Added padding
    plt.xlabel(eixo_x_label, fontsize=13)
    plt.ylabel('Produto (Descrição Resumida e Código SH4)', fontsize=13) # More descriptive ylabel
    plt.xticks(rotation=45, ha='right', fontsize=11) # Rotate x-ticks if values large
    plt.yticks(fontsize=11)
    plt.grid(axis='x', linestyle='--', alpha=0.6) # Add horizontal grid lines
    plt.tight_layout() # Adjust layout automatically to prevent labels overlapping
    plt.show() # Display the plot
    print(f"  Plot displayed successfully for {municipio_nome}.")


# --- Plot Top 5 Exporting Municipalities in SP ---
def criar_grafico_top5_municipios(dados, ano, metrica):
    """Generates and displays a bar chart for the top 5 exporting SP municipalities based on the selected metric."""
    print(f"--- Generating Plot: Top 5 SP Municipalities ---")
    print(f"  Year: {ano}")
    print(f"  Metric: {metrica}")

    # --- Validate Year and Data Availability ---
    if ano not in dados or dados[ano] is None or dados[ano].empty:
        errmsg = f"Erro: Dados não disponíveis para o ano {ano}."
        print(f"  {errmsg}")
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    df_ano = dados[ano]

    # --- Check if SP Municipality Name Data is Available ---
    if 'municipios_sp' not in globals() or municipios_sp.empty:
        errmsg = "Erro: Dados auxiliares de municípios ('municipios_sp') não disponíveis ou vazios.\n" \
                 "Não é possível adicionar nomes de municípios ao gráfico."
        # FIX: Perform replacement before f-string
        log_msg = errmsg.replace('\n', ' ')
        print(f"  {log_msg}")
        plt.figure(figsize=(10, 3))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return # Stop, as names are essential for this plot

    # --- Aggregate Data by Municipality based on Metric ---
    df_agregado = pd.DataFrame() # Initialize result DataFrame
    coluna_rank = None # This will hold the name of the column used for ranking
    eixo_x_label = '' # Plot x-axis label

    if metrica == 'Quilogramas':
        coluna_orig = 'KG_LIQUIDO'
        if coluna_orig not in df_ano.columns:
             print(f"  Error: Required column '{coluna_orig}' not found in data for {ano}.")
             return # Stop if essential column is missing
        # Group by municipality and sum the kilograms
        df_agregado = df_ano.groupby('CO_MUN')[coluna_orig].sum().reset_index()
        coluna_rank = coluna_orig # Rank by the sum of kilograms
        eixo_x_label = 'Total Exportado (Kg)'

    elif metrica == 'Valor Agregado':
        # Calculate overall average Valor Agregado for each municipality in the year
        if not all(col in df_ano.columns for col in ['VL_FOB', 'KG_LIQUIDO', 'CO_MUN']):
             print(f"  Error: Required columns ('VL_FOB', 'KG_LIQUIDO', 'CO_MUN') not found for {ano}.")
             return # Stop if essential columns missing

        # Group by municipality, summing FOB and KG
        df_agregado_temp = df_ano.groupby('CO_MUN').agg(
            Total_FOB=('VL_FOB', 'sum'),
            Total_KG=('KG_LIQUIDO', 'sum')
        ).reset_index()

        # Filter out municipalities with zero or near-zero total KG to avoid division errors
        epsilon = 1e-9
        df_agregado = df_agregado_temp[df_agregado_temp['Total_KG'] > epsilon].copy()

        if df_agregado.empty:
            infomsg = f"Sem municípios com exportações válidas (KG > 0) encontradas em {ano}\n" \
                      f"para calcular o Valor Agregado médio."
            # FIX: Perform replacement before f-string
            log_msg = infomsg.replace('\n', ' ')
            print(f"  Info: {log_msg}")
            plt.figure(figsize=(10, 4))
            plt.text(0.5, 0.5, infomsg, ha='center', va='center', fontsize=12, color='blue')
            plt.axis('off')
            plt.show()
            return

        # Calculate the average Valor Agregado for each municipality
        df_agregado['Valor_Agregado_Medio'] = df_agregado['Total_FOB'] / df_agregado['Total_KG']
        coluna_rank = 'Valor_Agregado_Medio' # Rank by this calculated average
        eixo_x_label = 'Valor Agregado Médio Municipal (USD/kg)'
    else:
        # Handle unknown metric
        print(f"  Error: Métrica desconhecida '{metrica}' para o gráfico Top 5 Municípios.")
        return


    # --- Rank and Get Top 5 ---
    # Check if the aggregation and rank column determination was successful
    if coluna_rank is None or coluna_rank not in df_agregado.columns:
         print(f"  Error Interno: Coluna de ranking '{coluna_rank}' não encontrada nos dados agregados.")
         return

    # Get the top 5 municipalities based on the ranking column
    top5_mun_codes = df_agregado.nlargest(5, coluna_rank)

    if top5_mun_codes.empty:
        infomsg = f"Não foi possível encontrar municípios para o ranking\n" \
                  f"Métrica: {metrica}\nAno: {ano}"
        # FIX: Perform replacement before f-string
        log_msg = infomsg.replace('\n', ' ')
        print(f"  Info: {log_msg}")
        plt.figure(figsize=(10, 4))
        plt.text(0.5, 0.5, infomsg, ha='center', va='center', fontsize=12, color='blue')
        plt.axis('off')
        plt.show()
        return

    # --- Merge with Municipality Names ---
    try:
         # Ensure merge keys ('CO_MUN' and 'CO_MUN_GEO') are compatible integer types
         top5_mun_codes['CO_MUN'] = top5_mun_codes['CO_MUN'].astype(int)
         # municipios_sp['CO_MUN_GEO'] should already be int from Cell 5 cleaning
         if 'CO_MUN_GEO' not in municipios_sp.columns:
             print("Error: 'CO_MUN_GEO' column missing in municipios_sp. Cannot merge names.")
             top5_mun_data = top5_mun_codes.copy()
             top5_mun_data['NO_MUN'] = 'Erro no Merge (Sem CO_MUN_GEO)'
         else:
             municipios_sp['CO_MUN_GEO'] = municipios_sp['CO_MUN_GEO'].astype(int) # Ensure type
             top5_mun_data = top5_mun_codes.merge(
                 municipios_sp[['CO_MUN_GEO', 'NO_MUN']], # Select only needed columns
                 left_on='CO_MUN',
                 right_on='CO_MUN_GEO',
                 how='left' # Keep all top 5 codes, even if name merge fails (indicates data issue)
             )
    except Exception as e:
         print(f"  Error merging Top 5 municipality codes with names: {e}")
         # Create a placeholder name column if merge fails to allow plotting codes
         top5_mun_data = top5_mun_codes.copy()
         top5_mun_data['NO_MUN'] = 'Erro no Merge - Código: ' + top5_mun_data['CO_MUN'].astype(str)

    # Handle potential missing names (if merge failed for some codes)
    top5_mun_data['NO_MUN'] = top5_mun_data['NO_MUN'].fillna('Nome Desconhecido')

    # --- Create the Plot ---
    plt.figure(figsize=(11, 7)) # Adjusted size

    # Sort data for consistent plotting order (highest value bar at the top)
    plot_data = top5_mun_data.sort_values(coluna_rank, ascending=True)

    # Create the horizontal bar plot using Seaborn
    sns.barplot(data=plot_data, y='NO_MUN', x=coluna_rank, color='mediumseagreen', orient='h') # Changed color

    # Customize plot appearance
    plt.title(f'Top 5 Municípios Exportadores SP ({ano}) por {metrica}', fontsize=16, pad=15)
    plt.xlabel(eixo_x_label, fontsize=13)
    plt.ylabel('Município', fontsize=13)
    plt.xticks(rotation=45, ha='right', fontsize=11)
    plt.yticks(fontsize=11)
    plt.grid(axis='x', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show() # Display the plot
    print(f"  Plot displayed successfully for Top 5 Municipalities.")


# --- Plot Comparison between TWO Municipalities ---
def criar_grafico_comparativo(dados, ano, metrica, municipio1_nome, municipio2_nome):
    """
    Generates and displays a grouped horizontal bar chart comparing the top 5 products
    (based on union of individual top 5s) of two selected municipalities.
    """
    print(f"--- Generating Plot: Municipality Comparison ---")
    print(f"  Municipality 1: {municipio1_nome}")
    print(f"  Municipality 2: {municipio2_nome}")
    print(f"  Year: {ano}")
    print(f"  Metric: {metrica}")

    # --- Input Validation ---
    if not municipio1_nome or not municipio2_nome or municipio1_nome == '-- N/A --' or municipio2_nome == '-- N/A --' or municipio1_nome == 'Select Municipality' or municipio2_nome == 'Select Municipality':
         errmsg = "Erro: Por favor, selecione dois municípios válidos para comparação."
         print(f"  {errmsg}")
         plt.figure(figsize=(10, 2))
         plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
         plt.axis('off')
         plt.show()
         return
    if municipio1_nome == municipio2_nome:
        errmsg = "Erro: Por favor, selecione dois municípios diferentes para comparação."
        print(f"  {errmsg}")
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    # --- Get Municipality Codes ---
    co_mun1 = get_co_mun(municipio1_nome)
    co_mun2 = get_co_mun(municipio2_nome)
    if co_mun1 is None or co_mun2 is None:
        # Error message handled by get_co_mun, create placeholder plot
        missing = []
        if co_mun1 is None: missing.append(f"'{municipio1_nome}'")
        if co_mun2 is None: missing.append(f"'{municipio2_nome}'")
        errmsg = f"Erro: Código não encontrado para: {', '.join(missing)}."
        print(f"  {errmsg}")
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    # --- Validate Year and Data Availability ---
    if ano not in dados or dados[ano] is None or dados[ano].empty:
        errmsg = f"Erro: Dados não disponíveis para o ano {ano}."
        print(f"  {errmsg}")
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    # --- Check if Product Description Data is Available ---
    if 'ncm_sh_df' not in globals() or ncm_sh_df.empty:
        errmsg = "Erro: Dados auxiliares de produtos ('ncm_sh_df') não disponíveis.\n" \
                 "Não é possível adicionar descrições de produtos ao gráfico."
        # FIX: Perform replacement before f-string
        log_msg = errmsg.replace('\n', ' ')
        print(f"  {log_msg}")
        plt.figure(figsize=(10, 3))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    # --- Calculate Metrics for ALL Products for Both Municipalities ---
    df_ano = dados[ano]
    metrica_col_calc = 'KG_LIQUIDO' if metrica == 'Quilogramas' else 'Valor_Agregado' # Name of calculated metric column
    eixo_x_label = 'Massa Líquida (Kg)' if metrica == 'Quilogramas' else 'Valor Agregado (USD/kg)'

    # Helper function to calculate metric for all products of one municipality
    def calculate_all_products(df_year, co_mun_sel, metric_name):
        # Filter for municipality
        df_mun_all = df_year[df_year['CO_MUN'] == co_mun_sel].copy()
        if df_mun_all.empty: return pd.DataFrame() # Return empty if no data rows

        metric_col = None # Will hold the name of the calculated metric column
        agg_df = pd.DataFrame() # Initialize

        if metric_name == 'Quilogramas':
             if 'KG_LIQUIDO' not in df_mun_all.columns: return pd.DataFrame() # Check required column
             agg_df = df_mun_all.groupby('SH4')['KG_LIQUIDO'].sum().reset_index()
             metric_col = 'KG_LIQUIDO'
        elif metric_name == 'Valor Agregado':
             if not all(c in df_mun_all.columns for c in ['VL_FOB', 'KG_LIQUIDO']): return pd.DataFrame() # Check cols
             agg_tmp = df_mun_all.groupby('SH4').agg(Total_FOB=('VL_FOB', 'sum'), Total_KG=('KG_LIQUIDO', 'sum')).reset_index()
             epsilon = 1e-9 # Use small epsilon for comparison
             agg_df = agg_tmp[agg_tmp['Total_KG'] > epsilon].copy() # Filter non-zero KG
             if agg_df.empty: return pd.DataFrame() # Return empty if no valid rows remain
             agg_df['Valor_Agregado'] = agg_df['Total_FOB'] / agg_df['Total_KG'] # Calculate metric
             metric_col = 'Valor_Agregado'
        else:
             print(f"  Error: Unknown metric '{metric_name}' in internal function.")
             return pd.DataFrame() # Invalid metric

        # Return DataFrame with SH4 and the calculated metric column, if successful
        if metric_col and metric_col in agg_df.columns and not agg_df.empty:
             return agg_df[['SH4', metric_col]].copy() # Return only SH4 and the metric value
        else:
             return pd.DataFrame() # Return empty on failure

    # Calculate for both municipalities
    mun1_agg = calculate_all_products(df_ano, co_mun1, metrica)
    mun2_agg = calculate_all_products(df_ano, co_mun2, metrica)

    # --- Check if Data Was Found for Calculation ---
    valid_mun1 = not mun1_agg.empty
    valid_mun2 = not mun2_agg.empty

    # Handle cases where one or both municipalities have no valid data for the metric
    if not valid_mun1 and not valid_mun2:
        infomsg = f"Sem dados válidos para a métrica '{metrica}' ({ano})\n" \
                  f"encontrados para NENHUM dos municípios:\n" \
                  f"{municipio1_nome} ou {municipio2_nome}."
        # FIX: Perform replacement before f-string
        log_msg = infomsg.replace('\n', ' ')
        print(f"  Info: {log_msg}")
        plt.figure(figsize=(10, 4))
        plt.text(0.5, 0.5, infomsg, ha='center', va='center', fontsize=12, color='blue')
        plt.axis('off')
        plt.show()
        return
    elif not valid_mun1:
         print(f"  Warning: Sem dados válidos encontrados para {municipio1_nome} (Métrica: {metrica}, Ano: {ano}). Comparação pode estar incompleta.")
    elif not valid_mun2:
         print(f"  Warning: Sem dados válidos encontrados para {municipio2_nome} (Métrica: {metrica}, Ano: {ano}). Comparação pode estar incompleta.")


    # --- Identify Top 5 Products (SH4 codes) for Each Municipality ---
    # Use the aggregated data (mun1_agg, mun2_agg) which contains the calculated metric column
    top5_sh4_mun1 = set(mun1_agg.nlargest(5, metrica_col_calc)['SH4']) if valid_mun1 else set()
    top5_sh4_mun2 = set(mun2_agg.nlargest(5, metrica_col_calc)['SH4']) if valid_mun2 else set()

    # --- Union of Top Products ---
    # Find all unique SH4 codes that appear in the top 5 of EITHER municipality
    union_sh4 = top5_sh4_mun1.union(top5_sh4_mun2)

    if not union_sh4:
        infomsg = f"Nenhum produto comum no Top 5 encontrado para\n" \
                  f"{municipio1_nome} e {municipio2_nome} ({ano})\n" \
                  f"Métrica: {metrica}"
        # FIX: Perform replacement before f-string
        log_msg = infomsg.replace('\n', ' ')
        print(f"  Info: {log_msg}")
        plt.figure(figsize=(10, 4))
        plt.text(0.5, 0.5, infomsg, ha='center', va='center', fontsize=12, color='blue')
        plt.axis('off')
        plt.show()
        return

    print(f"  Comparando com base em {len(union_sh4)} produto(s) únicos do Top 5 (Códigos SH4: {union_sh4})")

    # --- Prepare Data for Plotting ---
    # Filter the aggregated data for BOTH municipalities to include ONLY the products in the union set.
    # Add a 'Municipio' column to distinguish them.

    # Filter Muni 1 data
    if valid_mun1:
        mun1_plot_data = mun1_agg[mun1_agg['SH4'].isin(union_sh4)].copy()
        mun1_plot_data['Municipio'] = municipio1_nome
    else:
        # Create placeholder structure if no data for muni 1
        mun1_plot_data = pd.DataFrame({'SH4': list(union_sh4), metrica_col_calc: 0, 'Municipio': municipio1_nome})


    # Filter Muni 2 data
    if valid_mun2:
        mun2_plot_data = mun2_agg[mun2_agg['SH4'].isin(union_sh4)].copy()
        mun2_plot_data['Municipio'] = municipio2_nome
    else:
         # Create placeholder structure if no data for muni 2
        mun2_plot_data = pd.DataFrame({'SH4': list(union_sh4), metrica_col_calc: 0, 'Municipio': municipio2_nome})


    # --- Combine Data for Plotting ---
    # Concatenate the two dataframes
    combined_data = pd.concat([mun1_plot_data, mun2_plot_data], ignore_index=True)

    # --- Add Product Descriptions ---
    try:
        # Ensure merge keys are compatible types
        combined_data['SH4'] = combined_data['SH4'].astype(int)
        # ncm_sh_df['CO_SH4'] should be int from Cell 3
        if 'CO_SH4' not in ncm_sh_df.columns:
             print("Error: 'CO_SH4' column missing in ncm_sh_df. Cannot merge product names.")
             combined_data['NO_SH4_POR'] = 'Erro no Merge (Sem CO_SH4)'
        else:
             ncm_sh_df['CO_SH4'] = ncm_sh_df['CO_SH4'].astype(int) # Ensure type
             combined_data = combined_data.merge(
                 ncm_sh_df[['CO_SH4', 'NO_SH4_POR']],
                 left_on='SH4',
                 right_on='CO_SH4',
                 how='left' # Keep all products even if description is missing
             )
    except Exception as e:
        print(f"  Error merging combined comparison data with NCM SH names: {e}")
        combined_data['NO_SH4_POR'] = 'Erro no Merge' # Placeholder

    # Handle missing descriptions and create final 'Produto' label
    combined_data['NO_SH4_POR'] = combined_data['NO_SH4_POR'].fillna('Descrição Desconhecida')
    combined_data['Produto'] = combined_data['NO_SH4_POR'].astype(str).str[:40] + ' (' + combined_data['SH4'].astype(str) + ')'

    # Fill any missing metric values with 0 AFTER merge (e.g., if a product was top 5 for one muni but not exported by the other)
    combined_data[metrica_col_calc] = combined_data[metrica_col_calc].fillna(0)


    # --- Create the Grouped Bar Plot ---
    plt.figure(figsize=(14, max(6, len(union_sh4) * 0.9))) # Dynamic height based on # products

    # Define a color palette
    palette = {'viridis': sns.color_palette("viridis", n_colors=2),
               'paired': sns.color_palette("Paired", n_colors=2)}
    plot_palette = palette['viridis'] # Choose one

    # --- Optional: Order Products for Plot Clarity ---
    # Order by the maximum value achieved by either municipality for that product
    try:
        if not combined_data.empty and metrica_col_calc in combined_data.columns:
            # Group by product, find the max value across both municipalities, sort products by this max value
            product_order = combined_data.groupby('Produto')[metrica_col_calc].max().sort_values(ascending=False).index
            print(f"  Order of products in plot (descending by max value): {list(product_order)}")
        else:
            product_order = None # No specific order if calculation fails
    except Exception as e:
        print(f"  Warning: Could not determine product order for plot. Using default order. Error: {e}")
        product_order = None


    # Create the grouped horizontal bar plot using Seaborn
    sns.barplot(
        data=combined_data,
        y='Produto',
        x=metrica_col_calc,
        hue='Municipio', # This creates the grouping
        orient='h',
        order=product_order, # Apply the calculated order (or None)
        palette=plot_palette # Use defined palette
    )

    # Customize plot appearance
    plt.title(f'Comparação Top Produtos ({metrica}) - {municipio1_nome} vs {municipio2_nome} ({ano})', fontsize=16, pad=15)
    plt.xlabel(eixo_x_label, fontsize=13)
    plt.ylabel('Produto (Descrição Resumida e Código SH4)', fontsize=13) # More descriptive ylabel
    plt.xticks(rotation=45, ha='right', fontsize=11)
    plt.yticks(fontsize=11)
    plt.legend(title='Município', fontsize=11, title_fontsize=12, loc='lower right') # Adjust legend
    plt.grid(axis='x', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show() # Display the plot
    print(f"  Comparison plot displayed successfully.")


print("Cell 8/11: Plotting functions defined (Corrected).")

In [ ]:
# ===============================
# CELL 9: Event Handlers
# ===============================
# Define functions that will be called when widget values change or buttons are clicked.
# These functions will typically clear the output area and call the appropriate plotting function.

print("Defining event handler functions...")

# --- Handler for Dropdown Changes (Ano, Município Single, Métrica) ---
def on_dropdown_change(change):
    """
    Handles changes in the 'value' of ano_dropdown, municipio_dropdown, or metrica_dropdown.
    Updates the single municipality plot in the output_area.
    """
    # Check if the change notification is for the 'value' attribute
    if change['type'] == 'change' and change['name'] == 'value':
        # Use the 'output_area' context manager to direct output (plots, prints)
        with output_area:
            clear_output(wait=True) # Clear previous plot/messages before generating new one
            print(">>> Event: Dropdown changed. Updating single municipality plot...") # User feedback

            # Get the current values from the relevant dropdowns
            selected_ano = ano_dropdown.value
            selected_municipio = municipio_dropdown.value # This is the one for the single plot
            selected_metrica = metrica_dropdown.value

            # --- Validate Selections Before Plotting ---
            if not selected_ano or not selected_municipio or \
               selected_municipio in ['-- N/A --', 'Select Municipality']:
                errmsg = "Por favor, selecione um Ano e um Município válidos para visualizar o gráfico."
                print(f"  ! Validation Error: {errmsg}")
                # Display a message in the plot area instead of a plot
                plt.figure(figsize=(10, 2))
                plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='orange')
                plt.axis('off')
                plt.show()
                return # Stop execution for this handler

            # --- Call the Plotting Function ---
            # Pass the global data dictionary and current widget values
            criar_grafico_municipio_unico(
                dados=dados_ano,
                municipio_nome=selected_municipio,
                ano=selected_ano,
                metrica=selected_metrica
            )
            print("<<< Plot update complete.")


# --- Handler for Top 5 SP Municipalities Button Click ---
def on_top5_municipios_clicked(button_info): # The button object itself is passed
     """Handles clicks on the 'Top 5 Municípios Exportadores (SP)' button."""
     # Use the 'output_area' context manager
     with output_area:
        clear_output(wait=True) # Clear previous output
        print(">>> Event: 'Top 5 Municípios' button clicked. Generating plot...") # User feedback

        # Get current values from common filters
        selected_ano = ano_dropdown.value
        selected_metrica = metrica_dropdown.value

        # --- Validate Selections ---
        if not selected_ano:
             errmsg = "Por favor, selecione um Ano válido."
             print(f"  ! Validation Error: {errmsg}")
             plt.figure(figsize=(10, 2))
             plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='orange')
             plt.axis('off')
             plt.show()
             return # Stop

        # --- Call the Plotting Function ---
        criar_grafico_top5_municipios(
            dados=dados_ano,
            ano=selected_ano,
            metrica=selected_metrica
        )
        print("<<< Plot update complete.")


# --- Handler for Compare Municipalities Button Click ---
def on_compare_clicked(button_info):
    """Handles clicks on the 'Comparar Top 5 Produtos' button."""
    # Use the 'output_area' context manager
    with output_area:
        clear_output(wait=True) # Clear previous output
        print(">>> Event: 'Comparar Top 5 Produtos' button clicked. Generating comparison plot...") # User feedback

        # Get current values from common filters and comparison dropdowns
        selected_ano = ano_dropdown.value
        selected_metrica = metrica_dropdown.value
        selected_mun1 = municipio1_dropdown.value
        selected_mun2 = municipio2_dropdown.value

        # --- Call the Comparison Plotting Function ---
        # The validation logic is handled inside criar_grafico_comparativo
        criar_grafico_comparativo(
            dados=dados_ano,
            ano=selected_ano,
            metrica=selected_metrica,
            municipio1_nome=selected_mun1,
            municipio2_nome=selected_mun2
        )
        print("<<< Plot update complete.")


print("Cell 9/11: Event handler functions defined.")

In [ ]:
# ===============================
# CELL 10: Observe Widget Changes & Clicks
# ===============================
# Connect the widgets defined in Cell 6 to their respective event handler functions defined in Cell 9.

print("Attaching event handlers to widgets...")

# --- Observe Dropdown Value Changes ---
# When the 'value' of these dropdowns changes, call the 'on_dropdown_change' function.
# This handler updates the *single* municipality plot.
try:
    ano_dropdown.observe(on_dropdown_change, names='value')
    municipio_dropdown.observe(on_dropdown_change, names='value') # For the single view
    metrica_dropdown.observe(on_dropdown_change, names='value')
    print("  Observers attached to ano_dropdown, municipio_dropdown, metrica_dropdown.")
except NameError as e:
    print(f"  Warning: Could not attach observers. Widget likely not defined. Error: {e}")


# --- Assign Button Click Handlers ---
# When these buttons are clicked, call their respective handler functions.
try:
    top5_municipios_button.on_click(on_top5_municipios_clicked)
    print("  Click handler attached to top5_municipios_button.")
except NameError as e:
     print(f"  Warning: Could not attach click handler to top5_municipios_button. Error: {e}")

try:
    compare_button.on_click(on_compare_clicked)
    print("  Click handler attached to compare_button.")
except NameError as e:
     print(f"  Warning: Could not attach click handler to compare_button. Error: {e}")


print("\nCell 10/11: Widget observers and click handlers attached.")
print("--> Next cell will display the dashboard.")

In [ ]:
# ===============================
# CELL 11: Layout and Initial Display (Corrected)
# ===============================
# Arrange the interactive widgets using VBox (vertical) and HBox (horizontal) layout containers.
# Display the arranged widgets and the output area.
# Finally, trigger the display of the initial plot based on default widget values.

print("Arranging dashboard layout...")

# --- Define Layout Containers ---

# Container for common filters (Year, Metric)
common_filters = widgets.VBox([
    widgets.HTML("<b>Filtros Comuns:</b>", layout=widgets.Layout(margin='0 0 5px 0')), # Add bottom margin
    ano_dropdown,
    metrica_dropdown
], layout=widgets.Layout(margin='0 10px 0 0')) # Add right margin to the whole box

# Container for the single municipality view controls
controls_single = widgets.VBox([
    widgets.HTML("<b>Visualizar por Município:</b>", layout=widgets.Layout(margin='0 0 5px 0')),
    municipio_dropdown
    # Note: The plot updates automatically when municipio_dropdown changes (via observer)
], layout=widgets.Layout(margin='0 10px 0 0')) # Add right margin

# Container for the "Top 5 SP Municipalities" button
controls_top_mun = widgets.VBox([
     widgets.HTML("<b>Visualizar Top Municípios (SP):</b>", layout=widgets.Layout(margin='15px 0 5px 0')), # Add top margin for spacing
     top5_municipios_button
     # Uses Ano and Métrica selected in common_filters
])

# Container for the comparison controls
controls_compare = widgets.VBox([
    widgets.HTML("<b>Comparar Municípios:</b>", layout=widgets.Layout(margin='15px 0 5px 0')), # Add top margin
    municipio1_dropdown,
    municipio2_dropdown,
    compare_button
    # Uses Ano and Métrica selected in common_filters
])

# --- Arrange the Containers ---
# Example 2: Group controls logically in two columns
left_column = widgets.VBox([common_filters, controls_single])
right_column = widgets.VBox([controls_top_mun, controls_compare])
dashboard_controls = widgets.HBox([left_column, right_column])


print("Layout arranged.")

# --- Display the Dashboard ---
print("\n--- Displaying Dashboard ---")
display(dashboard_controls)
print("--- Output Area (Plots will appear below) ---")
display(output_area) # Display the output area where plots will render

# --- Trigger the Initial Plot ---
# This runs *after* the widgets are displayed.
# Use the 'with output_area' context to ensure the initial plot appears inside it.
with output_area:
    # clear_output(wait=True) # Ensure it's clear before the first plot
    print("--- Generating Initial Plot (based on defaults) ---")

    # Get default values from widgets to generate the first view
    initial_ano = ano_dropdown.value
    initial_municipio = municipio_dropdown.value
    initial_metrica = metrica_dropdown.value

    # Check if the default values are valid before attempting to plot
    if initial_ano and initial_municipio and \
       initial_municipio not in ['-- N/A --', 'Select Municipality']:
        # If defaults are valid, call the single municipality plot function
        criar_grafico_municipio_unico(
            dados=dados_ano,
            municipio_nome=initial_municipio,
            ano=initial_ano,
            metrica=initial_metrica
        )
    else:
        # If defaults are not valid (e.g., data loading failed), display a message
        warning_msg = "Não foi possível gerar o gráfico inicial.\n" \
                      "Verifique se os dados foram carregados corretamente (Células 3-5)\n" \
                      "e se há opções válidas selecionadas nos menus."
        # FIX: Perform replacement before f-string
        log_msg = warning_msg.replace('\n', ' ')
        print(f"  ! Info: {log_msg}")
        # Display the message within the output area using a simple plot
        plt.figure(figsize=(10, 3))
        plt.text(0.5, 0.5, warning_msg, ha='center', va='center', fontsize=12, color='orange')
        plt.axis('off')
        plt.show()

print("\nCell 11/11: Dashboard displayed. Initial plot generated (if possible).")
print("==========================================")
print(">>> Dashboard is ready to use! <<<")
print("==========================================")

In [ ]:
def clean_metadata():
    display(Javascript('''
    try {
      const md = JSON.parse(JSON.stringify(IPython.notebook.metadata));
      if (md.widgets) {
        console.log('Limpando metadados de widgets...');
        delete md.widgets;
      }
      IPython.notebook.metadata = md;
      IPython.notebook.save();
    } catch(err) {
      console.log('Nenhum widget persistente encontrado');
    }
    '''))

# Execute esta função antes de exportar
clean_metadata()